# Clase Práctica 08

# CNN - Redes Neuronales Convolucionales

# Clasificación multiclase con CNN

En esta clase práctica se revisará la forma de cargar una base de datos para realizar la clasificación de imágenes. Para realizar esta tarea, se utilizará un código programado en python, con la librería glob para cargar las imagenes almacenadas en una carpeta. La carpeta debe ser organizada mediante subcarpetas cuyos nombres corresponden a las clases que se quieren clasificar. Una vez que se obtienen los conjuntos de imágenes de validación y test, se procede a crear un modelo de red neuronal convolucional básico para realizar la tarea de clasificación.

#  Importando las imágenes 

Como de costumbre, debemos cargar las librerias necesarias para poder realizar las tareas de clasificación. En esta parte del código, se muestra como cargar las imagenes de una base de datos. Las imágenes vienen en diferentes directorios (carpetas) y usamos el nombre de cada directorio como nombre de la clase. El código asigna un identificador numérico a cada clase, es decir si hay 5 directorios (carpetas), nuestro código asigna 5 clases (clase: 0, 1, 2, 3, 4). 

In [ ]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from keras.utils import np_utils
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


# parámetros
num_classes = 5
img_rows, img_cols = 100, 100
input_shape = (img_rows, img_cols,3)

# función para cargar las imágenes 

def load_data(path, formato):
    class_names={}
    class_id=0
    Xx = []
    Yy = []
    for d in glob.glob(os.path.join(path, '*')):
        clname = os.path.basename(d)
        for f in glob.glob(os.path.join(d, formato)): 
            if not clname in class_names:
                class_names[clname]=class_id 
                class_id += 1
            img = image.load_img(f, target_size=(img_rows, img_cols))
            npi = image.img_to_array(img)       
            #npi = preprocess_input(npi)
            Xx.append(npi)
            Yy.append(class_names[clname])
    return np.array(Xx), np.array(Yy), class_names


#  Cargando la base de datos

Para cargar las imágenes de la base de datos, llamamos a la función load_data creada en el apartado anterior. Acá asignamos la ruta y el nombre del directorio que contiene las imágenes, además del tipo de datos. Para revisar si se cargaron las imágenes de manera correcta se realiza una visualización de una imagen mediante python. 

Otra revisión importante es analizar la cantidad de datos obtenidos por nuestra función. Note que la forma de los datos corresponden a (tamaño total de las muestras, dimensión x, dimensión y, profundidad).  

In [ ]:
# cargar las imágenes, etiquetas y nombres de las clases
Xx, Yy, class_names = load_data('flower_photos', '*.jpg')
#Xx, Yy, class_names = load_data('profesores', '*.png')

num_classes = len(class_names)

print("Las siguientes imágenes fueron exportadas (ejemplos, fila, col, prof) :")
print(Xx.shape)
print("Las siguientes etiquetas y clases fueron exportadas:")
print(Yy.shape, len(class_names))

# Revisando las imágenes cargadas

Note que la función plot solo muestra valores entre 0 y 1 adecuadamente. Se observa que la imágen se ve extraña. La razón es la imagen no está en el rango correcto. 

In [ ]:
plt.imshow(Xx[0,:,:,:])

In [ ]:
i = Xx[0,:,:,:]
print("Valores mínimos y máximos de la imagen")
maxi=(np.max(i))
mini=(np.min(i))
print(mini,maxi )

Vamos a corregir la imágen con una pequeña normalización de rango (0,1). Ahora la imagen se ve como corresponde.

In [ ]:
i2 = (i-mini)/(maxi-mini)
plt.imshow(i2)

Creemos una función para mostrar la imagen corregida

In [ ]:
def normalizar_imagen(imagen):
    maxi=(np.max(imagen))
    mini=(np.min(imagen))
    i2 = (imagen-mini)/(maxi-mini)
    plt.imshow(i2)

normalizar_imagen(Xx[0,:,:,:])

#  Dividiendo las imágenes

En esta parte del código se realiza la división de las imágenes en el conjunto de entrenamiento y test. Utilizamos la función train_test_split para separar los datos. Además, se utiliza la conversión a categorical para obtener una matriz con las clases de manera binaria.   

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Xx, Yy, test_size=0.1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

x_train = x_train.astype('float32')/ 255
x_val = x_val.astype('float32') / 255 
x_test = x_test.astype('float32') /255

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape[0], 'train labels')
print(y_val.shape[0], 'val labels')
print(y_test.shape[0], 'test labels')

YY_train=y_train # respaldar variables originales sin hot encoding
YY_val=y_val
YY_test=y_test
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
normalizar_imagen(x_train[0])

#  Modelo de CNN

A continuación se define el modelo a utilizar, en este caso un modelo propio y no uno pre entrenado como vimos en el tutorial anterior. 

Se realizan pruebas de varios modelos de red convolucional para hacer las pruebas de clasificación. Note que por medio de la función model.summary() es posible visualizar el modelo completo que será entrenado con las imagenes de la base de datos propia. 

In [ ]:
from keras.layers.normalization import BatchNormalization
# modelo
def modelo_cnn():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.50))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = modelo_cnn()

In [ ]:
model.summary()

#  Ajustar la red neuronal convolucional 

En esta parte del código se realiza el entrenamiento de la red neuronal. Note que se guarda la historia del entrenamiento para poder visualizar el aprendizaje de la red. Recordar que se quiere obtener una minimización de la perdida por lo cual se espera que converja a números cercanos a cero. Además se desea que la validación sea de una manera similar a la curva de entrenamiento, es decir que exista convergencia a números pequeños del loss.  

Como resultado nos entrega el error de clasificación y nos entrega la precisión obtenida de los datos de test. 

In [ ]:
batch_size = 20
epochs = 20

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

# Validación de la predicción

A continuación se detalla como realizar la validación de la predicción mediante una imagen de test. Luego se realiza la predicción multiple de varias imagenes de test con su respectiva clasificación. Para realizar las tareas de predicción, se ejecuta el comando model.predict(). Utilizando el argmax() es posible obtener la clase correspondiente a la imagen de entrada. 

In [ ]:
import numpy as np
index = np.random.choice(list(range(len(x_test))), 1)[0]
im = x_test[index]
target_class = {0:"Daisy",1:"Dandelion",2:"Roses",3:"Sunflowers",4:"Tulips"}
#target_class = {0:"Esteban",1:"Gabriel",2:"Gonzalo",3:"Héctor",4:"Seba"}

print('la imagen de test:')
normalizar_imagen(im)

print('la clase que predice la red es: ', target_class[np.argmax(model.predict(np.reshape(im, [1,img_rows, img_cols,3])), -1)[0]])

In [ ]:
import numpy as np
# predicciones de nuestra CNN con los datos de test
predicted_classes = model.predict_classes(x_test)

# revisemos cuales predicciones son correctas e incorrectas
correct_indices = np.nonzero(predicted_classes == YY_test)[0]
incorrect_indices = np.nonzero(predicted_classes != YY_test)[0]


plt.figure(figsize=(10,6))
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    normalizar_imagen(x_test[correct])
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], YY_test[correct]))

plt.figure(figsize=(10,6))
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    normalizar_imagen(x_test[incorrect])
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], YY_test[incorrect]))

# Matriz de confusión y reportes

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score 

# classification report
print('Reporte de Clasificación:')
print(classification_report( YY_test,predicted_classes))

# confusion matrix 
cm = confusion_matrix(YY_test,predicted_classes)

# mostrar los resultados
print('Matriz de confusión:')
print(cm)

# Print f1, precision, and recall scores
print('Precision:')
print(precision_score(YY_test, predicted_classes , average="macro"))
print('Recall:')
print(recall_score(YY_test, predicted_classes , average="macro"))
print('F1:')
print(f1_score(YY_test, predicted_classes, average="macro"))